## Callback Playground

Notebook that uses `fmi` callbacks

In [7]:
from fastai.vision.all import *
from fastai.medical.imaging import *
from fmi.train import *
from fmi.pipeline import *

In [8]:
pneu = untar_data(URLs.SIIM_SMALL)
p_items = get_dicom_files(f'{pneu}/train')
df = pd.read_csv(f'{pneu}/labels.csv')

In [9]:
splitter = RandomSplitter(valid_pct=0.1)(p_items)
item_tfms = Resize(266)
batch_tfms = [ Normalize.from_stats(*imagenet_stats)]

pneumothorax = DataBlock(blocks=(ImageBlock(cls=DicomView), CategoryBlock),
                   get_x=lambda x:pneu/f"{x[0]}",
                   get_y=lambda x:x[1],
                   splitter=RandomSplitter(valid_pct=0.2),
                   item_tfms = item_tfms,
                   batch_tfms = batch_tfms)

dls = pneumothorax.dataloaders(df[:20].values, 
                               bs=6, 
                               num_workers=0,
                               drop_last = False,
                               get_idxs = None)

In [10]:
dls.train_ds

(#16) [(DicomView mode=L size=1024x1024, TensorCategory(1)),(DicomView mode=L size=1024x1024, TensorCategory(0)),(DicomView mode=L size=1024x1024, TensorCategory(0)),(DicomView mode=L size=1024x1024, TensorCategory(0)),(DicomView mode=L size=1024x1024, TensorCategory(0)),(DicomView mode=L size=1024x1024, TensorCategory(1)),(DicomView mode=L size=1024x1024, TensorCategory(0)),(DicomView mode=L size=1024x1024, TensorCategory(0)),(DicomView mode=L size=1024x1024, TensorCategory(1)),(DicomView mode=L size=1024x1024, TensorCategory(0))...]

Lets see how many times each class is present in `dls.train_ds`

In [37]:
classes = [dls.vocab[i[-1].item()] for i in dls.train_ds]
Counter(classes)

Counter({'Pneumothorax': 5, 'No Pneumothorax': 11})

**EpochIteration** - displays `Epoch` and `Iteration` with the option to display images

**EpochImageCounter** - counts the number of images in each class after each training epoch

In [17]:
learn = vision_learner(dls, 
                       'resnet18' , 
                       metrics=[accuracy, error_rate], 
                       cbs=[EpochIteration(show_img=False),
                           EpochImageCounter(dls)])

Train DS Size: 16
Batch Size: 6


In [14]:
learn.fit(2)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.442853,0.619244,0.750000,0.250000,00:00
1,1.091550,0.645191,0.750000,0.250000,00:00


Training: Epoch: 0 Iter: 0 Loss:0.0
Training: Epoch: 0 Iter: 1 Loss:1.9728779792785645
Training: Epoch: 0 Iter: 2 Loss:0.5880052447319031
Counter({'No Pneumothorax': 11, 'Pneumothorax': 5})
Validation: Epoch: 0 Iter: 0 Loss:1.7715716361999512
Training: Epoch: 1 Iter: 0 Loss:0.6192443370819092
Training: Epoch: 1 Iter: 1 Loss:0.9267821311950684
Training: Epoch: 1 Iter: 2 Loss:0.7162213325500488
Counter({'No Pneumothorax': 22, 'Pneumothorax': 10})
Validation: Epoch: 1 Iter: 0 Loss:0.6453918814659119


With each epoch as expected 11 `No Pneumothorax` and 5 `Pneumothorax` images pass throught the model and after 2 epochs a total of 22 `No Pneumothorax` and 10 `Pneumothorax` images pass through the model

By default `drop_last` is set to `False` in `dataloaders`.  This means that *all* images pass through the model with each epoch. However if you change this to `True` and using `EpochImageCounter` you can see how this affects how many images of each class pass through the model. 

In [42]:
dls = pneumothorax.dataloaders(df[:20].values, 
                               bs=6, 
                               num_workers=0,
                               drop_last = True,
                               get_idxs = None)

learn = vision_learner(dls, 
                       'resnet18' , 
                       metrics=[accuracy, error_rate], 
                       cbs= EpochImageCounter(dls))
learn.fit(2)

Train DS Size: 16
Batch Size: 6


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.035917,5.148357,0.250000,0.750000,00:00
1,1.112166,4.216203,0.250000,0.750000,00:00


Counter({'No Pneumothorax': 8, 'Pneumothorax': 4})
Counter({'No Pneumothorax': 17, 'Pneumothorax': 7})


With `drop_last` set to `True` and `batch_size` of 6 you would expect 2 iterations (2 * 6 = 12) hence each iteration would have 2 images less that the total number of images in `dls.train_ds` which is 16.